In [2]:
!pip install citipy
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
# Import API key
from api_keys import g_key

C:\Users\danie\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
url= 'http://cluebase.lukelav.in/contestants?limit=1000&order_by=total_winnings&sort=desc'

In [4]:
response = requests.get(url)
response_json= response.json()
data=response_json['data']
data

[{'id': 208,
  'name': 'Ken Jennings',
  'notes': 'a software engineer from Salt Lake City, Utah',
  'games_played': 94,
  'total_winnings': 2522700},
 {'id': 75,
  'name': 'James Holzhauer',
  'notes': 'a professional sports gambler from Las Vegas, Nevada',
  'games_played': 33,
  'total_winnings': 2464216},
 {'id': 204,
  'name': 'David Madden',
  'notes': 'a student originally from Ridgewood, New Jersey',
  'games_played': 29,
  'total_winnings': 432400},
 {'id': 220,
  'name': 'Julia Collins',
  'notes': 'a supply chain professional from Kenilworth, Illinois',
  'games_played': 32,
  'total_winnings': 429100},
 {'id': 209,
  'name': 'Matt Jackson',
  'notes': 'a paralegal from Washington, D.C.',
  'games_played': 27,
  'total_winnings': 413612},
 {'id': 214,
  'name': 'Austin Rogers',
  'notes': 'a bartender from New York, New York',
  'games_played': 26,
  'total_winnings': 413000},
 {'id': 2117,
  'name': 'Arthur Chu',
  'notes': 'a compliance analyst and voiceover artist from Br

In [5]:
desc=[]
winnings=[]
games=[]
name=[]
for each in data:
    desc.append(each['notes'])
    winnings.append(each['total_winnings'])
    games.append(each['games_played'])
    name.append(each['name'])
winnings

[2522700,
 2464216,
 432400,
 429100,
 413612,
 413000,
 298200,
 267002,
 236405,
 231200,
 223597,
 215900,
 201000,
 187199,
 186900,
 182001,
 181401,
 177534,
 172202,
 171902,
 166603,
 164721,
 157202,
 157001,
 154800,
 151802,
 149570,
 149404,
 148597,
 147700,
 147603,
 147598,
 147403,
 144602,
 142804,
 142600,
 142200,
 141100,
 139803,
 139100,
 139088,
 138300,
 137300,
 136801,
 136000,
 134900,
 134402,
 130804,
 130100,
 129902,
 128998,
 128404,
 127002,
 126500,
 125797,
 125600,
 125597,
 124300,
 123705,
 123302,
 122802,
 122802,
 122800,
 118099,
 117803,
 117800,
 117102,
 116601,
 116503,
 116501,
 115800,
 114998,
 113600,
 113299,
 112000,
 110411,
 110200,
 109605,
 109499,
 109049,
 109002,
 109000,
 107602,
 107400,
 107204,
 107000,
 105902,
 105803,
 105500,
 105497,
 105203,
 105103,
 104800,
 104300,
 104205,
 104103,
 104000,
 103597,
 103002,
 102600,
 101200,
 101102,
 100400,
 100201,
 100054,
 99800,
 98699,
 98690,
 98599,
 98002,
 97598,
 9720

In [6]:
location=[]
loc=[]
for each in desc:
    x=each.split('from ')
    location.append(x[1])
for each in location:
    x=each.split('(')
    for each in x:
        y=each.split(')',1)
        for each in y:
            z=each.split(',')
            loc.append(z)


In [10]:
loca=pd.DataFrame(loc, columns=['city','state','other'])
win=pd.DataFrame(winnings, columns= ['amount'])
win
loca['amount']=win
loca['name']=pd.DataFrame(name, columns= ['name'])
loca['notes']=pd.DataFrame(desc, columns= ['notes'])
loca['games']=pd.DataFrame(games, columns= ['games'])
loca

,city,state,other,amount,name,notes,games
0,Salt Lake City,Utah,None,2522700.0,Ken Jennings,"a software engineer from Salt Lake City, Utah",94.0
1,Las Vegas,Nevada,None,2464216.0,James Holzhauer,"a professional sports gambler from Las Vegas, ...",33.0
2,Ridgewood,New Jersey,None,432400.0,David Madden,"a student originally from Ridgewood, New Jersey",29.0
3,Kenilworth,Illinois,None,429100.0,Julia Collins,"a supply chain professional from Kenilworth, I...",32.0
4,Washington,D.C.,None,413612.0,Matt Jackson,"a paralegal from Washington, D.C.",27.0
...,...,...,...,...,...,...,...
1027,Pataskala,Ohio,None,NaN,NaN,NaN,NaN
1028,St. Charles,Illinois,None,NaN,NaN,NaN,NaN
1029,Washington,D.C.,None,NaN,NaN,NaN,NaN
1030,Cleveland,Ohio,None,NaN,NaN,NaN,NaN


In [11]:
params = {"key": g_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in loca.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location= row['city']
    state= row['state']

    # update address key value
    params['address'] = f"{location},{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    try:
        cities_lat_lng = cities_lat_lng.json()
        loca.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        loca.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except:
        print('N/A')
# Visualize to confirm lat lng appear
loca.head()

N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A


,city,state,other,amount,name,notes,games,Lat,Lng
0,Salt Lake City,Utah,None,2522700.0,Ken Jennings,"a software engineer from Salt Lake City, Utah",94.0,40.760779,-111.891047
1,Las Vegas,Nevada,None,2464216.0,James Holzhauer,"a professional sports gambler from Las Vegas, ...",33.0,36.171563,-115.139101
2,Ridgewood,New Jersey,None,432400.0,David Madden,"a student originally from Ridgewood, New Jersey",29.0,40.979264,-74.116531
3,Kenilworth,Illinois,None,429100.0,Julia Collins,"a supply chain professional from Kenilworth, I...",32.0,42.085862,-87.717561
4,Washington,D.C.,None,413612.0,Matt Jackson,"a paralegal from Washington, D.C.",27.0,38.907192,-77.036871


In [21]:
loca['Lat']=loca["Lat"].astype(str)
loca['Lng']=loca["Lng"].astype(str)
loca['location'] = loca['Lat'] + "," + loca['Lng']     
locat=loca[loca.Lat!="nan"]
df=locat.drop(columns='other')
df=df.dropna()
df2=df.head(200)
df2

,city,state,amount,name,notes,games,Lat,Lng,location
0,Salt Lake City,Utah,2522700.0,Ken Jennings,"a software engineer from Salt Lake City, Utah",94.0,40.7607793,-111.8910474,"40.7607793,-111.8910474"
1,Las Vegas,Nevada,2464216.0,James Holzhauer,"a professional sports gambler from Las Vegas, ...",33.0,36.171563,-115.1391009,"36.171563,-115.1391009"
2,Ridgewood,New Jersey,432400.0,David Madden,"a student originally from Ridgewood, New Jersey",29.0,40.9792645,-74.11653129999999,"40.9792645,-74.11653129999999"
3,Kenilworth,Illinois,429100.0,Julia Collins,"a supply chain professional from Kenilworth, I...",32.0,42.085862,-87.71756069999999,"42.085862,-87.71756069999999"
4,Washington,D.C.,413612.0,Matt Jackson,"a paralegal from Washington, D.C.",27.0,38.9071923,-77.0368707,"38.9071923,-77.0368707"
...,...,...,...,...,...,...,...,...,...
199,Santa Barbara,California,75000.0,John Ryan,"a student from Santa Barbara, California",8.0,34.4208305,-119.6981901,"34.4208305,-119.6981901"
200,Athens,Georgia,74701.0,Sara Wilkinson,"a country club concierge from Athens, Georgia",4.0,33.9519347,-83.357567,"33.9519347,-83.357567"
201,Brooklyn,New York,74601.0,Rob Liguori,"a research editor from Brooklyn, New York",5.0,40.6781784,-73.9441579,"40.6781784,-73.9441579"
202,New York,New York,74600.0,Jeff Goldfarb,"a recent law school graduate from New York, Ne...",3.0,40.7127753,-74.0059728,"40.7127753,-74.0059728"


In [22]:
df2.to_json("C:/Users/danie/Jeo-PARTY/static/js/location.json", orient='records')

In [19]:
import locationtagger
city=[]
state[]
for each in df['notes']:
    try:
        place_entity = locationtagger.find_locations(text = each)
        city.append(place_entity.cities)
        state.append(place_entity.states)
    except:
        print('N/A') 
    print(place_entity.cities)

['Salt Lake City']
['Las Vegas', 'Nevada']
['Ridgewood']
['Kenilworth']
['Washington']
['New York']
['Broadview Heights', 'Ohio']
['Chicago']
['Washington', 'Seattle']
['Newark', 'Delaware']
['California', 'El Cerrito']
['Highland Park']
['Somerville']
['Minneapolis']
['Washington']
['Austin']
['Steamboat Springs', 'Colorado']
['Florence']
['Amherst', 'Ohio']
['Boston']
['Los Angeles', 'California']
['North Little Rock', 'North']
['Minneapolis']
['New York', 'Brooklyn']
['Moscow']
['Chicago']
['Milan', 'Michigan']
['New York', 'South Dayton']
['Charlotte']
['Mount Airy']
['Manchester', 'New Hampshire']
['Aurora', 'Colorado']
['St. Louis']
['New York', 'Amherst']
['Ohio']
['Eastpointe', 'Michigan']
['Berwyn']
['Buford']
['Minneapolis']
['Ohio']
['New York']
['Albuquerque']
['Las Vegas']
['Baltimore', 'Maryland']
['Edina']
['Orlando', 'Florida']
['Annapolis', 'Maryland']
['Durham', 'New Hampshire']
['Washington']
['Gillette']
['Malden']
['California', 'Studio City']
['Ohio', 'Boardman']


['Chicago']
['Bristol']
['New York']
['Pennington']
['California', 'Van Nuys', 'English']
['Nashville', 'Tennessee']
['Kansas City']
['Atlanta']
['Manchester', 'Vermont']
['Orlando', 'Florida']
['California', 'North Hills', 'English']
['Murfreesboro', 'Tennessee']
['California', 'Altadena']
['Boston']
['California', 'Stockton']
['Bethesda', 'Maryland', 'University']
['Sacramento', 'California']
['Philadelphia']
[]
['Carrollton']
[]
['Abiquiu']
['Lexington']
[]
['Washington', 'Vancouver']
['Vancouver', 'Canada']
['Lincolnshire']
['Burlington', 'Vermont']
['New York', 'Croton']
['Bethesda', 'Maryland']
['Kansas City']
['Toronto', 'Ontario', 'Canada']
['University']
['Carmel']
['Randolph']
['Stoughton']
['California', 'Berkeley']
['Toronto', 'Ontario']
['Los Angeles', 'California']
['Los Angeles', 'California']
['Elmhurst']
['Arlington Heights']
['New Orleans', 'Louisiana']
['Minneapolis']
['Austin']
['Pasadena', 'California']
['Gainesville']
['Los Angeles', 'California']
['Rockford']
['G

['Fort Worth']
['Hilliard', 'Ohio']
['Cincinnati', 'Ohio']
['St. Paul']
['Oak Ridge', 'Tennessee']
['New York']
['Boston']
['Los Angeles', 'California']
['Atlanta']
['Virginia', 'Norfolk']
['Washington', 'Seattle']
['Decatur']
['Cincinnati', 'Ohio']
['New Orleans', 'Louisiana']
['New York', 'White Plains']
['New York', 'Lockport']
['Cranston']
['California', 'Ventura']
['Douglasville']
['Fremont', 'California']
['New York', 'Sunnyside']
['Westminster', 'Maryland']
['East Lansing', 'Michigan']
['McAlester']
['Prairie Village', 'Kansas']
['California', 'Palo Alto']
['Sharon']
['California', 'Twentynine Palms']
['Arnold']
['Epping', 'New Hampshire']
['Washington', 'Seattle']
['Woodhaven', 'Michigan']
['Grinnell', 'Iowa']
['California', 'Simi Valley']
['Silver Spring', 'Maryland']
['Millersville', 'Maryland']
['California', 'Altadena']
['Chicago']
['Birmingham']
['Mount Laurel']
['Virginia', 'Arlington']
['Highland Park']
['New York']
['California', 'Pacifica']
['Virginia', 'Midlothian']
[